<a href="https://colab.research.google.com/github/glwat/Durham_Masters/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gethin Watkins Master's - Data Cleaning, Aggregation and Compilation



This script forms part of the Watkins master's series. It takes rockfall inventory and climate observations compliled by previous authors at Whitby and produces a master dataset, comprising total rockfall counts and volumes, alongside a range of envrionmental observations at hourly intervals.

The script combines individual rockfall observations taken by previous authors with temperature and reflectance data captured within Ignasio's PhD.   

Codeblock 001:  Mount the Google Drive

In [1]:
# Codeblock 001 — Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Codeblock 002: Import the individual rockfall inventory from the google drive. This dataset comprises individual observations of rockfall, and the corresponding rainfall, air temperature and humidity measurements at the hour in which the rockfall occurred. The structure of the data means each occurring rockfall comprises its own observation in the dataframe - the maximum event count allowed in each observation is 1. Hours in which no rockfall occurred are also included, meaning  rainfall, humidity and air temperature observations in hours without rockfall are available.

In [2]:
# Codeblock 002 — Import file as DataFrame and show variable info
import pandas as pd

# Replace this path with your file path inside Google Drive
file_path = "/content/drive/MyDrive/Masters_Colab_Data/Rockfall_Dataset_2021_Amended.xlsx"

# Load the dataset
df = pd.read_excel(file_path)

# Display basic variable info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11407 entries, 0 to 11406
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    11407 non-null  int64  
 1   Datetime              11407 non-null  object 
 2   Gap_Rockfall          8223 non-null   object 
 3   Gap_hours             2022 non-null   float64
 4   Rockfall_Volume       8223 non-null   float64
 5   Volume_Error          4105 non-null   float64
 6   Rockfall_Event_Count  4105 non-null   float64
 7   Total_Rainfall        11406 non-null  float64
 8   Air_Temp              11407 non-null  float64
 9   Relative_Humidity     11407 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 891.3+ KB


Codeblock 004: The Datetime column is confirmed as datetime format. The dataset is then aggregated to ensure one only one observation is present for each hour in the timeframe. Rockfall volumes, event counts and volume errors are summed to give the total volume and number of occurrences for each hour.

In [3]:
# Codeblock 004 — Datetime conversion, aggregation, and summary counts

# Ensure Datetime column is a proper datetime dtype
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')

# Pre‑aggregation stats
n_obs_before = len(df)
n_unique_datetimes = df['Datetime'].nunique()

print("Number of observations before aggregation:", n_obs_before)
print("Number of unique datetime values:", n_unique_datetimes)

# Define aggregation rules
agg_rules = {
    'ID': 'first',
    'Datetime': 'first',
    'Gap_Rockfall': 'first',
    'Gap_hours': 'first',
    'Rockfall_Volume': 'sum',
    'Volume_Error': 'sum',
    'Rockfall_Event_Count': 'sum',
    'Total_Rainfall': 'first',
    'Air_Temp': 'first',
    'Relative_Humidity': 'first'
}

# Perform aggregation
df_agg = df.groupby('Datetime', as_index=False).agg(agg_rules)

# Post‑aggregation stats
n_obs_after = len(df_agg)

print("Number of observations after aggregation:", n_obs_after)

# Optional: quick structural check
df_agg.info()

Number of observations before aggregation: 11407
Number of unique datetime values: 8377
Number of observations after aggregation: 8377
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8377 entries, 0 to 8376
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    8377 non-null   int64         
 1   Datetime              8377 non-null   datetime64[ns]
 2   Gap_Rockfall          5193 non-null   object        
 3   Gap_hours             258 non-null    float64       
 4   Rockfall_Volume       8377 non-null   float64       
 5   Volume_Error          8377 non-null   float64       
 6   Rockfall_Event_Count  8377 non-null   float64       
 7   Total_Rainfall        8376 non-null   float64       
 8   Air_Temp              8377 non-null   float64       
 9   Relative_Humidity     8377 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
mem

Codeblock 005: Rainfall observations are provided in a total rainfall format, corresponding to total rainfall since the previous midnight (exclusive of midnight; the value for midnight indicates the rainfall over the previous 24hrs). This codeblock generates a new variable Hourly_Rainfall that corresponds to the rainfall occurring in each hour. There were a few issues in the dataset - hourly rainfall could return as negative due to errors in the input dataframe. These are set to NA in the codeblock. To generate the hourly rainfall, the code takes the Total_Rainfall value if the time is 1AM. In all other instances, it takes the total rainfall away from the total rainfall of an hour prior, unless data is missing. In this case NA is returned. Data gaps exists, with missing hours. This is accounted for.  

In [4]:
# Codeblock 005 — Generate Hourly_Rainfall variable (updated with negative-value rule)

import numpy as np

# Ensure dataframe is sorted by datetime
df_agg = df_agg.sort_values('Datetime').reset_index(drop=True)

# Extract hour for rule logic
df_agg['Hour'] = df_agg['Datetime'].dt.hour

# Shift Total_Rainfall by 1 hour (aligned by row order)
df_agg['Prev_Total_Rainfall'] = df_agg['Total_Rainfall'].shift(1)
df_agg['Prev_Datetime'] = df_agg['Datetime'].shift(1)

# Check whether previous row is exactly one hour earlier
df_agg['One_Hour_Prior'] = (df_agg['Datetime'] - df_agg['Prev_Datetime']) == pd.Timedelta(hours=1)

def compute_hourly_rainfall(row):
    # Rule 1: If hour == 1AM → Hourly_Rainfall = Total_Rainfall
    if row['Hour'] == 1:
        return row['Total_Rainfall']

    # Rule 2: For all other hours:
    # Must have a valid previous observation exactly 1 hour prior
    if not row['One_Hour_Prior']:
        return np.nan

    # Missing rainfall in either observation → NA
    if pd.isna(row['Total_Rainfall']) or pd.isna(row['Prev_Total_Rainfall']):
        return np.nan

    # Compute difference
    hr = row['Total_Rainfall'] - row['Prev_Total_Rainfall']

    # NEW RULE: If Hourly_Rainfall < 0 (but not equal to zero), return NA
    if hr < 0:
        return np.nan

    return hr

df_agg['Hourly_Rainfall'] = df_agg.apply(compute_hourly_rainfall, axis=1)

# Optional: clean helper columns
# df_agg = df_agg.drop(columns=['Hour', 'Prev_Total_Rainfall', 'Prev_Datetime', 'One_Hour_Prior'])

df_agg[['Datetime', 'Total_Rainfall', 'Hourly_Rainfall']].head()

,Datetime,Total_Rainfall,Hourly_Rainfall
0,2021-01-01 00:00:00,NaN,NaN
1,2021-01-01 01:00:00,0.00,0.00
2,2021-01-01 02:00:00,0.51,0.51
3,2021-01-01 03:00:00,0.51,0.00
4,2021-01-01 04:00:00,0.51,0.00


In [5]:
# Codeblock 006 — Generate variable quality summary dataframe

import numpy as np
import pandas as pd

# List variables you want to check
vars_of_interest = [
    'Rockfall_Volume',
    'Volume_Error',
    'Rockfall_Event_Count',
    'Total_Rainfall',
    'Hourly_Rainfall',
    'Air_Temp',
    'Relative_Humidity'
]

summary_rows = []

for var in vars_of_interest:
    series = df_agg[var]

    summary_rows.append({
        'Variable': var,
        'Min': series.min(skipna=True),
        'Max': series.max(skipna=True),
        'Non-NA Count': series.notna().sum(),
        'NA Count': series.isna().sum(),
        'Count < 0': (series < 0).sum()
    })

# Create summary dataframe
df_summary = pd.DataFrame(summary_rows)

df_summary

,Variable,Min,Max,Non-NA Count,NA Count,Count < 0
0,Rockfall_Volume,0.0,101.865240,8377,0,0
1,Volume_Error,0.0,34.430918,8377,0,0
2,Rockfall_Event_Count,0.0,266.000000,8377,0,0
3,Total_Rainfall,0.0,48.010000,8376,1,0
4,Hourly_Rainfall,0.0,12.190000,8330,47,0
5,Air_Temp,-3.1,33.700000,8377,0,217
6,Relative_Humidity,36.5,100.000000,8377,0,0


#Combining Temperature Dataset

Codeblock 010: Loads the temperature observations of Whitby cliff into a new dataframe.  

In [6]:

# Codeblock 010 — Load Excel file into df_temp

import pandas as pd

# Replace this path with the actual filepath to your Excel file
temp_file_path = "/content/drive/MyDrive/Masters_Colab_Data/Temperature_Data_2021_Edit_ABSOLUTE.xlsx"

# Load the Excel file
df_temp = pd.read_excel(temp_file_path)

# Quick structural check
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Temp_ID             8760 non-null   int64         
 1   Date                8760 non-null   datetime64[ns]
 2   Hour                8760 non-null   object        
 3   Thermal_image_name  4276 non-null   object        
 4   TA                  4276 non-null   float64       
 5   Tmean               4276 non-null   float64       
 6   Tmin                4276 non-null   float64       
 7   Tmax                4276 non-null   float64       
 8   Tmeandiff           4276 non-null   float64       
 9   Tmindiff            4276 non-null   float64       
 10  Tmaxdiff            4276 non-null   float64       
 11  W1-BSS-Tmean        4276 non-null   float64       
 12  W2-IBSS-Tmean       4276 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1), obje

Codeblock 011: Combines the Date and Hour variables of the temperature dataset into a new Datetime variable, removing the Date and Hour variables afterwards.

In [7]:
# Codeblock 011 — Combine Date and Hour into a single Datetime variable and drop originals

import pandas as pd

# Ensure Date and Hour are strings (avoids Excel auto-format issues)
df_temp['Date'] = df_temp['Date'].astype(str)
df_temp['Hour'] = df_temp['Hour'].astype(str)

# Combine into a single datetime column
df_temp['Datetime'] = pd.to_datetime(
    df_temp['Date'] + " " + df_temp['Hour'],
    errors='coerce'
)

# Drop original columns
df_temp = df_temp.drop(columns=['Date', 'Hour'])

# Quick check
df_temp[['Datetime']].head()
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Temp_ID             8760 non-null   int64         
 1   Thermal_image_name  4276 non-null   object        
 2   TA                  4276 non-null   float64       
 3   Tmean               4276 non-null   float64       
 4   Tmin                4276 non-null   float64       
 5   Tmax                4276 non-null   float64       
 6   Tmeandiff           4276 non-null   float64       
 7   Tmindiff            4276 non-null   float64       
 8   Tmaxdiff            4276 non-null   float64       
 9   W1-BSS-Tmean        4276 non-null   float64       
 10  W2-IBSS-Tmean       4276 non-null   float64       
 11  Datetime            8760 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 821.4+ KB


Codeblock 012: Count the number of missing observations (i.e. the number of hours missing from the dataset) in the temperature dataset

In [8]:
# Codeblock 012 — Count missing hourly timestamps in df_temp

import pandas as pd

# Ensure Datetime is sorted
df_temp = df_temp.sort_values('Datetime').reset_index(drop=True)

# Define the full expected hourly range
full_range = pd.date_range(
    start=df_temp['Datetime'].min(),
    end=df_temp['Datetime'].max(),
    freq='H'
)

# Convert to a set for fast comparison
actual_times = set(df_temp['Datetime'])
expected_times = set(full_range)

# Identify missing timestamps
missing_times = sorted(list(expected_times - actual_times))

# Report counts
print("First timestamp:", df_temp['Datetime'].min())
print("Last timestamp:", df_temp['Datetime'].max())
print("Total expected hours:", len(full_range))
print("Actual observations:", len(df_temp))
print("Missing hourly timestamps:", len(missing_times))

# Optional: show the missing timestamps
# missing_times[:20]  # preview first 20

First timestamp: 2021-01-01 00:00:00
Last timestamp: 2021-12-31 23:00:00
Total expected hours: 8760
Actual observations: 8760
Missing hourly timestamps: 0


/tmp/ipython-input-338541123.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(


Codeblock 013: Insert new observations with NA across all variables (apart from Datetime) to account for missing hours and to allow the later join of datasets

In [9]:
# Codeblock 013 — Insert missing hourly observations with NA values

import pandas as pd
import numpy as np

# Ensure df_agg is sorted by datetime
df_agg = df_agg.sort_values('Datetime').reset_index(drop=True)

# Create full expected hourly range
full_range = pd.date_range(
    start=df_agg['Datetime'].min(),
    end=df_agg['Datetime'].max(),
    freq='H'
)

# Reindex the dataframe to the full hourly range
df_full = df_agg.set_index('Datetime').reindex(full_range)

# Restore Datetime as a column
df_full = df_full.rename_axis('Datetime').reset_index()

# All newly created rows will have NA in all other columns automatically

# Quick check
print("Original number of rows:", len(df_agg))
print("New number of rows (after filling gaps):", len(df_full))
print("Number of inserted missing-hour rows:", len(df_full) - len(df_agg))

Original number of rows: 8377
New number of rows (after filling gaps): 8760
Number of inserted missing-hour rows: 383


/tmp/ipython-input-861837486.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(


Codeblock 014: Join the rockfall inventory with the temperature dataset to create a new joined dataframe comprising observations from both datasets. Check whether this join has been successful.

In [10]:
# Codeblock 014 — Join df_agg and df_temp on Datetime and generate join report

import pandas as pd

# Ensure both dataframes are sorted and Datetime is datetime dtype
df_agg['Datetime'] = pd.to_datetime(df_agg['Datetime'], errors='coerce')
df_temp['Datetime'] = pd.to_datetime(df_temp['Datetime'], errors='coerce')

df_agg = df_agg.sort_values('Datetime').reset_index(drop=True)
df_temp = df_temp.sort_values('Datetime').reset_index(drop=True)

# Perform the join (left join keeps all rows from df_agg)
df_joined = df_agg.merge(df_temp, on='Datetime', how='left', indicator=True)

# Generate join report
total_rows = len(df_joined)
matched_rows = (df_joined['_merge'] == 'both').sum()
unmatched_left = (df_joined['_merge'] == 'left_only').sum()
unmatched_right = (df_joined['_merge'] == 'right_only').sum()  # should be 0 for left join

print("JOIN REPORT")
print("-----------")
print("Total rows in joined dataframe:", total_rows)
print("Rows with matching Datetime in both datasets:", matched_rows)
print("Rows in df_agg with NO match in df_temp:", unmatched_left)
print("Rows in df_temp with NO match in df_agg:", unmatched_right)

# Optional: inspect unmatched rows
# df_joined[df_joined['_merge'] != 'both'].head()

# Drop merge indicator if no longer needed
# df_joined = df_joined.drop(columns=['_merge'])

JOIN REPORT
-----------
Total rows in joined dataframe: 8377
Rows with matching Datetime in both datasets: 8377
Rows in df_agg with NO match in df_temp: 0
Rows in df_temp with NO match in df_agg: 0


#Combine Reflectivity Data With The Joined Dataset

Codeblock 015: Load the excel file containing reflectivity observations at Whitby cliff into a new dataframe.

In [11]:


# Codeblock 015 — Load another Excel file and report dataset info + row count

import pandas as pd

# Replace this with the actual filepath to your Excel file
new_file_path = "/content/drive/MyDrive/Masters_Colab_Data/Reflectivity_data_2021.xls"

# Load the Excel file
df_new = pd.read_excel(new_file_path)

# Display structural information
df_new.info()

# Report number of observations
print("\nNumber of observations in df_new:", len(df_new))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5140 entries, 0 to 5139
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Ref_File   5140 non-null   object 
 1   DateTime   5140 non-null   object 
 2   Ref_Mean   5140 non-null   float64
 3   Ref_Max    5140 non-null   float64
 4   Ref_Min    5140 non-null   float64
 5   Ref_Range  5140 non-null   float64
 6   Ref_sd     5140 non-null   float64
dtypes: float64(5), object(2)
memory usage: 281.2+ KB

Number of observations in df_new: 5140


Codeblock 017: Remove any duplicate hours (i.e. hours with identical datetime measurements

In [12]:
# Codeblock 017 — Remove duplicate DateTime rows (keep first) and fill missing hours

import pandas as pd
import numpy as np

# Ensure DateTime is parsed correctly
df_new['DateTime'] = pd.to_datetime(df_new['DateTime'], errors='coerce')

# Sort by DateTime
df_new = df_new.sort_values('DateTime').reset_index(drop=True)

# Step 1 — Count and report duplicates
duplicate_count = df_new['DateTime'].duplicated().sum()
print("Number of duplicate DateTime rows removed:", duplicate_count)

# Step 2 — Remove duplicates (keep first occurrence)
df_new = df_new.drop_duplicates(subset='DateTime', keep='first')

# Step 3 — Create full expected hourly range
full_range_new = pd.date_range(
    start=df_new['DateTime'].min(),
    end=df_new['DateTime'].max(),
    freq='h'   # lowercase to avoid FutureWarning
)

# Step 4 — Reindex to full hourly range
df_new_full = df_new.set_index('DateTime').reindex(full_range_new)

# Step 5 — Restore DateTime as a column
df_new_full = df_new_full.rename_axis('DateTime').reset_index()

# Step 6 — Report
print("Original number of rows after deduplication:", len(df_new))
print("New number of rows after filling gaps:", len(df_new_full))
print("Number of inserted missing-hour rows:", len(df_new_full) - len(df_new))

df_new_full.head()

Number of duplicate DateTime rows removed: 9
Original number of rows after deduplication: 5131
New number of rows after filling gaps: 8759
Number of inserted missing-hour rows: 3628


,DateTime,Ref_File,Ref_Mean,Ref_Max,Ref_Min,Ref_Range,Ref_sd
0,2021-01-01 01:00:00,210101_014658.tif,-10.777718,-5.037747,-15.939015,10.901268,1.395919
1,2021-01-01 02:00:00,210101_024707.tif,-10.970958,-5.366537,-15.783996,10.417459,1.396375
2,2021-01-01 03:00:00,210101_034720.tif,-10.861423,-5.287881,-15.820294,10.532413,1.411564
3,2021-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 05:00:00,210101_054727.tif,-12.052334,-6.755850,-15.874676,9.118825,1.342531


Codeblock 018: Join the reflectivity dataframe with our already joined dataset to create a master dataframe with temperature and reflectivity observations included alongside hourly rockfall volumes and numbers.   

In [13]:
# Codeblock 18 — Robust join using detected timestamp columns

import pandas as pd

# --- Step 1: Detect timestamp column in df_joined ---
possible_time_cols = ['Datetime', 'DateTime', 'datetime', 'date_time']

join_col_joined = None
for col in possible_time_cols:
    if col in df_joined.columns:
        join_col_joined = col
        break

if join_col_joined is None:
    raise ValueError("No datetime-like column found in df_joined. Columns are: " + str(df_joined.columns))

# --- Step 2: Detect timestamp column in df_new_full ---
join_col_new = None
for col in possible_time_cols:
    if col in df_new_full.columns:
        join_col_new = col
        break

if join_col_new is None:
    raise ValueError("No datetime-like column found in df_new_full. Columns are: " + str(df_new_full.columns))

# --- Step 3: Standardise both to 'Datetime' ---
df_joined = df_joined.rename(columns={join_col_joined: 'Datetime'})
df_new_full = df_new_full.rename(columns={join_col_new: 'Datetime'})

# Parse timestamps
df_joined['Datetime'] = pd.to_datetime(df_joined['Datetime'], errors='coerce')
df_new_full['Datetime'] = pd.to_datetime(df_new_full['Datetime'], errors='coerce')

# --- Step 4: Remove leftover merge indicators ---
for df in [df_joined, df_new_full]:
    if '_merge' in df.columns:
        df.drop(columns=['_merge'], inplace=True)

# --- Step 5: Sort for cleanliness ---
df_joined = df_joined.sort_values('Datetime').reset_index(drop=True)
df_new_full = df_new_full.sort_values('Datetime').reset_index(drop=True)

# --- Step 6: Perform the join ---
df_master = df_joined.merge(df_new_full, on='Datetime', how='left', indicator=True)

# --- Step 7: Join report ---
print("JOIN REPORT")
print("-----------")
print("Total rows:", len(df_master))
print("Matched rows:", (df_master['_merge'] == 'both').sum())
print("Left-only rows:", (df_master['_merge'] == 'left_only').sum())
print("Right-only rows:", (df_master['_merge'] == 'right_only').sum())

JOIN REPORT
-----------
Total rows: 8377
Matched rows: 8376
Left-only rows: 1
Right-only rows: 0


Codeblock 019: Drop any unmatched rows to account for a misalignment of 1hr between the start of the reflectivity and already joined dataframe.   

In [14]:
# Codeblock 019 — Safely drop unmatched row and clean df_master

# Drop the unmatched row only if the merge indicator exists
if '_merge' in df_master.columns:
    df_master = df_master[df_master['_merge'] != 'left_only'].reset_index(drop=True)
    df_master = df_master.drop(columns=['_merge'])
else:
    print("No _merge column found — it was already removed earlier.")

print("df_master cleaned. New number of rows:", len(df_master))
print("Earliest timestamp in df_master:", df_master['Datetime'].min())
print("Latest timestamp in df_master:", df_master['Datetime'].max())

df_master cleaned. New number of rows: 8376
Earliest timestamp in df_master: 2021-01-01 01:00:00
Latest timestamp in df_master: 2021-12-31 23:00:00


In [15]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8376 entries, 0 to 8375
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    8376 non-null   int64         
 1   Datetime              8376 non-null   datetime64[ns]
 2   Gap_Rockfall          5193 non-null   object        
 3   Gap_hours             258 non-null    float64       
 4   Rockfall_Volume       8376 non-null   float64       
 5   Volume_Error          8376 non-null   float64       
 6   Rockfall_Event_Count  8376 non-null   float64       
 7   Total_Rainfall        8376 non-null   float64       
 8   Air_Temp              8376 non-null   float64       
 9   Relative_Humidity     8376 non-null   float64       
 10  Hour                  8376 non-null   int32         
 11  Prev_Total_Rainfall   8375 non-null   float64       
 12  Prev_Datetime         8376 non-null   datetime64[ns]
 13  One_Hour_Prior    

Codeblock 020: Drop any variables which we have created to produce the joins or nolonger have a need for

In [16]:
# Codeblock 020 — Drop unused variables and summarise remaining columns safely

import pandas as pd
import numpy as np

# Columns to drop
cols_to_drop = [
    'Hour',
    'Prev_Total_Rainfall',
    'Prev_Datetime',
    'One_Hour_Prior',
    'Temp_ID'
]

# Drop only if they exist
df_master = df_master.drop(columns=[c for c in cols_to_drop if c in df_master.columns])

print("Dropped columns:", [c for c in cols_to_drop if c in df_master.columns])

# Prepare summary dataframe
summary = pd.DataFrame(index=df_master.columns)

# Missing values
summary['missing_values'] = df_master.isna().sum()

# Min values (numeric + datetime only)
summary['min'] = df_master.apply(
    lambda col: col.min() if np.issubdtype(col.dtype, np.number) or np.issubdtype(col.dtype, np.datetime64) else None
)

# Max values (numeric + datetime only)
summary['max'] = df_master.apply(
    lambda col: col.max() if np.issubdtype(col.dtype, np.number) or np.issubdtype(col.dtype, np.datetime64) else None
)

summary

Dropped columns: []


,missing_values,min,max
ID,0,2,11407
Datetime,0,2021-01-01 01:00:00,2021-12-31 23:00:00
Gap_Rockfall,3183,None,None
Gap_hours,8118,0.0,24.0
Rockfall_Volume,0,0.0,101.86524
Volume_Error,0,0.0,34.430918
Rockfall_Event_Count,0,0.0,266.0
Total_Rainfall,0,0.0,48.01
Air_Temp,0,-3.1,33.7
Relative_Humidity,0,36.5,100.0


Codeblock 021a: Generate binary rockfall occurrence indicator

In [20]:
# Codeblock 021a — Add Rockfall_Occurrence variable

# Create a binary indicator: 1 if any rockfall event occurred, else 0
df_master['Rockfall_Occurrence'] = (df_master['Rockfall_Event_Count'] >= 1).astype(int)

print("Rockfall_Occurrence variable added successfully.")
print(df_master[['Datetime', 'Rockfall_Event_Count', 'Rockfall_Occurrence']].head())

df_master.info()

Rockfall_Occurrence variable added successfully.
             Datetime  Rockfall_Event_Count  Rockfall_Occurrence
0 2021-01-01 01:00:00                   0.0                    0
1 2021-01-01 02:00:00                   0.0                    0
2 2021-01-01 03:00:00                   0.0                    0
3 2021-01-01 04:00:00                   0.0                    0
4 2021-01-01 05:00:00                   0.0                    0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8376 entries, 0 to 8375
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    8376 non-null   int64         
 1   Datetime              8376 non-null   datetime64[ns]
 2   Gap_Rockfall          5193 non-null   object        
 3   Gap_hours             258 non-null    float64       
 4   Rockfall_Volume       8376 non-null   float64       
 5   Volume_Error          8376 non-null   float

Codeblock 021: Save the new master dataframe containing a combination of merged datasets to an excel file in the google drive

In [18]:
# Codeblock 021 — Save df_master to Google Drive

from google.colab import drive
import pandas as pd

# Mount Google Drive (safe to run even if already mounted)
drive.mount('/content/drive')

# Define output path
output_path = '/content/drive/MyDrive/Masters_Colab_Data/Master_Rockfall_2021.xlsx'

# Save df_master to Excel
df_master.to_excel(output_path, index=False)

print("df_master saved to:", output_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
df_master saved to: /content/drive/MyDrive/Masters_Colab_Data/Master_Rockfall_2021.xlsx
